In [1]:
import os
# Set the number of processors to be used via environment variables
os.environ['OMP_NUM_THREADS'] = '10'
os.environ['TF_NUM_INTRAOP_THREADS'] = '10'
os.environ['TF_NUM_INTEROP_THREADS'] = '10'

In [2]:
import pickle
import polars as pl
import pandas as pd
from glob import glob

kdprov = "32"
data_imputation=glob(f'/data/ksa/04_Data_Preprocessing/{kdprov}/01_imputation/*_imputed_data.pkl')
bridging_date=pd.read_excel("/data/ksa/03_Sampling/bridging.xlsx", dtype='object', sheet_name="periode_to_date")
bridging_ksa=pd.read_excel("/data/ksa/03_Sampling/bridging.xlsx", dtype='object')

ls_dt=[]
for j in data_imputation:
    with open(j,'rb') as file:
        y=pickle.load(file)
        y['idsubsegment']=y['idpoint'].apply(lambda y1: y1.split('#')[0])
        y=pl.from_pandas(y)
        ls_dt.append(y)
df=pl.concat(ls_dt).to_pandas()
df['periode_start']=df.periode.apply(lambda y: y.split('_')[0][4:])
df=df.sort_values(['idsubsegment','periode']).merge(bridging_ksa.query('is_kabisat==0')[['periode_start','obs_in_a_year']])
df['tahun']=df.periode.apply(lambda y:str(y.split('_')[0][2:4]))

df['bulan']=df['obs_in_a_year'].astype(str)
df['VV'] = df['Sigma0_VV_db_imputation']
df['VH'] = df['Sigma0_VH_db_imputation']

df_label = pd.read_csv("/data/raw/processed/relabelled_data_ksa.csv",dtype='object')
df_label['idsubsegment']=df_label['id_x']
df = df[['periode','idsubsegment','tahun','bulan','VV','VH']].merge(
    df_label[['idsubsegment','tahun','bulan','obs']])
df2 = df.query('obs not in ["0.0","6.0","12.0"]')
#df2['segment'] = df2.idsubsegment.str[:9]

df2['idss'] = df2['idsubsegment']+df2['periode']
df2var = df2[['idss','VV','VH']]
df2var = df2var.groupby(df2var.idss).agg(["var"]).reset_index()
df2var.columns = df2var.columns.droplevel(1)
df2var.rename({'VV': 'Var_VV', 'VH': 'Var_VH'}, axis=1)
df2var = df2var.rename({'VV': 'Var_VV', 'VH': 'Var_VH'}, axis=1)
df2a = df2.merge(df2var, on = 'idss', how ='left')

/tmp/ipykernel_2029880/1045350448.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['idss'] = df2['idsubsegment']+df2['periode']


In [7]:
df2a['sequence']=df2a.sort_values(['Var_VH'],ascending=False).groupby('obs').cumcount()

In [6]:
df2a['sequence']=df2a.sort_values(['Var_VH'],ascending=False).groupby('obs').cumcount()

ValueError: Cannot set a DataFrame with multiple columns to the single column countg

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import datasets

from sklearn_som.som import SOM

# Extract just two features (just for ease of visualization)
som_class = df2[['VV','VH']].values

# Build a 3x1 SOM (3 clusters)
som = SOM(m=5, n=5, dim=2, random_state=1234)

# Fit it to the data
som.fit(som_class)

# Assign each datapoint to its predicted cluster
df2['som_val'] = som.predict(som_class)

In [ ]:
df

In [ ]:
pd.crosstab(df_c2['som_val'], df_c2['obs'])

In [ ]:
annual_income = df['Annual Income (k$)'].tolist()
spending_score = df['Spending Score (1-100)'].tolist()
fig = plt.figure(figsize=(6,6))
plt.scatter(annual_income, spending_score)
plt.suptitle("Scatter Plot of Annual Income & Spending Score")
plt.xlabel("Annual Income (k$)")
plt.ylabel("Spending Score (1-100)")
plt.show()


In [ ]:
from minisom import MiniSom  
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# I use linear som topography
som_shape = (1, 5)

som = MiniSom(som_shape[0], som_shape[1], som_class.shape[1], sigma=0.5, learning_rate=0.5)

max_iter = 1000
q_error = []
t_error = []

for i in range(max_iter):
    rand_i = np.random.randint(len(som_class))
    som.update(som_class[rand_i], som.winner(som_class[rand_i]), i, max_iter)
    q_error.append(som.quantization_error(som_class))
    t_error.append(som.topographic_error(som_class))

plt.plot(np.arange(max_iter), q_error, label='quantization error')
plt.plot(np.arange(max_iter), t_error, label='topographic error')
plt.ylabel('Quantization error')
plt.xlabel('Iteration index')
plt.legend()
plt.show()